### Tutorial_Implement_New_Analytical_Metric.ipynb

Victor Zhang and Daniel Pook-Kolb

# Summary: 

Jupyter Notebook Tutorial demonstrating how to implement a new analytical metric in MOTSFinder. There are numerous other ways to put a new metric into MOTSFinder, but the approach outlined below is the simplest option to get started. 

We demonstrate by showing the steps in implementing the Kerr metric in Cartesian Kerr-Schild form. Before doing so, we comment on two basic potential usages of MOTSFinder and their respective necessary quantities.

### Usage 1: 

For calculating quantities on the slice (e.g. spectrum for the stability operator of [3], etc.), you will need:
- an analytic expression for the 3-metric and extrinsic curvature of the slice/hypersurface

### Usage 2: 

For time evolution purposes, or any other purpose involving multiple slices/hypersurfaces, you will need:
- an analytic expression for the 3-metric, lapse, shift, and extrinsic curvature of the slice/hypersurface
- (Optional) analytic expressions for the derivatives (any nonzero order) of the shift, lapse, and extrinsic curvature of the slice/hypersurface
    
Clearly, Usage 1 is included in Usage 2, and so in the following tutorial, we assume your purpose is Usage 2 (i.e. we implement analytic expressions for the 3-metric, lapse, shift, and extrinsic curvature).

# <span style="color:red">Important:</span>
- You should always perform convergence tests with different resolutions for the finite difference method in taking derivatives and pseudo-spectral method for curves. This will help you estimate the accuracy of your numerical results. To avoid redundancy, convergence tests are not included below.
    - The parameter ```res``` denotes the inverse finite difference step size for taking derivatives. Change this to vary the resolution for the finite difference method.
    - The parameter ```num``` denotes the number of basis functions to use when expanding a function in a Fourier-like basis (i.e. into Sines, Cosines, Chebyshev polynomials, etc.). Change this to vary the resolution of the pseudo-spectral method. 

# Other Notes:
- MOTSFinder assumes axisymmetry; each 2D plot drawn is the $xz$ plane
- Points are in Cartesian $(x,y,z)$ coordinates; thus you cannot implement an analytical metric with angular coordinates in MOTSFinder
- Triple quotes ```"""..."""``` in the code are for Doxygen to generate documentation. Adding documentation (including references to where formulas come from) is strongly recommended for clarity and ease of use for yourself and others.
- We take $G = c = \hbar = 1$ and use Latin indices, e.g. $i,j,k$, to denote spacelike coordinates ($x,y,z$) and Greek indices, e.g. $\alpha,\beta,\mu,\nu$, to denote spacetime coordinates ($t,x,y,z$).


### References: 

[1] Matzner, Richard A., Mijan F. Huq, and Deirdre Shoemaker. "Initial
    data and coordinates for multiple black hole systems." Physical Review
    D 59.2 (1998): 024015.

[2] Moreno, Claudia, $Dar\acute{i}o$ $N\acute{u}\tilde{n}ez$, and Olivier Sarbach. "Kerr-
    Schild type initial data for black holes with angular momenta". Class.
    Quant. Grav. 19 (2002) 6059-6073.
    
[3] Andersson, Lars, Marc Mars, and Walter Simon. "Stability of marginally outer trapped 
    surfaces and existence of marginally outer trapped tubes." arXiv preprint arXiv:0704.2889 (2007).

# Step 1: Make a copy of the file ```schwarzschildks.py```, and rename

## Comments: 
- On Mac and Linux, duplicating the file ```schwarzschildks.py``` can be done in at least two ways:
    - Via the terminal. Enter and run this command from the terminal: 
        - ```cp ~/src/distorted-motsfinder-public/motsfinder/metric/analytical/schwarzschildks.py ~/src/distorted-motsfinder-public/motsfinder/metric/analytical/metricX.py```
    - Manually: Open ```schwarzschildks.py``` in your favorite text or code editor/IDE, and copy and paste the entire contents to a new file, and save the new file under a different name in the folder ```distorted-motsfinder-public/motsfinder/metric/analytical/```
- Rename the duplicated file to something other than ```schwarzschildks.py```. For simplicity, we assume throughout this tutorial that the new file name is ```metricX.py```, and we refer to the name of the new analytical metric to be implemented in MOTSFinder as the ```metricX``` metric
- Your new analytical metric should be implemented in ```metricX.py```. 

### Notes before starting the formal implementation:
- All steps from this step forth are in order of appearance in the code in the newly created ```metricX.py```, unless otherwise stated. If you followed the previous step, ```metricX.py``` should currently contain exactly the same code as ```schwarzschildks.py```. 

- Descriptive comments are assumed to be added/edited appropriately by the user as the tutorial progresses.

# Step 2: Rename for Organization

## a)  In the first line of ```metricX.py```, change ```motsfinder.metric.analytical.schwarzschildks``` to ```motsfinder.metric.analytical.metricX```

For clarity, the lines of code around this are:
```python
r"""@package motsfinder.metric.analytical.schwarzschildks

Schwarzschild slice in Kerr-Schild coordinates.

The class defined here can be used to produce the data for a slice of
Schwarzschild spacetime in Kerr-Schild form.
"""
```

## b) 

A few lines below, in the line

```python
__all__ = [
    "SchwarzschildKSSlice",
]
```

Change 
```python 
"SchwarzschildKSSlice"
``` 
to 
```python
"MetricXSlice"
```


# Step 3: Creating the ```MetricXSlice```

## a) Change the Class Name

Modify

```python
class SchwarzschildKSSlice(_ThreeMetric):
    r"""Data for a slice of Schwarzschild spacetime in Kerr-Schild coordinates.

    The formulas implemented here are based on [1].

    ...
```

to 

```python
class MetricXSlice(_ThreeMetric):
    # Edit comments as appropriate
```

## b) (i) Add parameters in ```__init__```, if needed

Add parameters/constants needed for your ```metricX``` metric in ```__init__``` in the ```MetricXSlice``` class. For example, if adding the Kerr metric in Kerr-Schild form, you would make the changes:

Schwarzschild (in the ```SchwarzschildKSSlice``` class):
```python
def __init__(self, M=1):
    r"""Create a Schwarzschild slice in Kerr-Schild form.

    @param M
        ADM mass of the spacetime.
    """
    super().__init__()
    self._M = float(M)
```

Kerr (in the ```KerrKSSlice``` class):
```python
def __init__(self, M=1, a=0):
    r"""Create a Kerr slice in Kerr-Schild form.

    @param M
        ADM mass of the spacetime.
    @param a
        Angular momentum parameter of the black hole.
    """
    super().__init__()
    self._M = float(M)
    self._a = float(a)
```

Note that we added the angular momentum parameter, $a$, with optional comments above.

## b) (ii) Create a method for any parameters added, if any

If you added a parameter in ```__init__```, add a method (using ```def```) to access the parameter when a ```MetricXSlice``` object is created.

In the Kerr case, you would add (in the ```KerrKSSlice``` class):

```python
@property
def a(self):
    r"""Angular momentum parameter of the black hole."""
    return self._a
```

The line ```@property``` is syntax in Python denoting the method immediately following to be accessible as a property (i.e. you can access it using the call ```g.a``` instead of ```g.a()```, if the instance of your object was ```g```).

## c) Define the 3-metric in ```_mat_at```

Enter your analytical form for the 3-metric (metric on the spacelike hypersurface) into the method ```_mat_at```. 

In the Kerr case, the 3-metric in Kerr-Schild form is, from [1]:

$$ g_{ij} = \delta_{ij} + 2Hl_il_j $$

where 

$$ H = \frac{Mr}{r^2 + a^2\cos^2\theta}$$

and 

$$ l_{\mu} = \left(1, \frac{rx+ay}{r^2 + a^2}, \frac{ry - ax}{r^2 + a^2}, \frac{z}{r}\right)$$

Notes (see [1] for more details):
- $\delta_{ij}$ is the Kronecker delta
- $M$ is the mass of the Kerr black hole, $a$ is the angular momentum parameter
- $z = r\cos\theta$; We use Cartesian Kerr-Schild coordinates, so we must replace $\cos\theta$ with $\frac{z}{r}$. 
- $\frac{x^2+y^2}{r^2+a^2} + \frac{z^2}{r^2} = 1$
- $r^2 = \frac{1}{2}\left(\rho^2 - a^2\right) + \sqrt{\frac{1}{4}\left(\rho^2 - a^2\right) + a^2z^2}$
- $\rho = \sqrt{x^2 + y^2 + z^2}$

Thus, in ```_mat_at```, we write:

```python
def _mat_at(self, point):
    r"""Three metric.

    This is eq. (15) of [1], using same notation.

    @b References

    [1] Matzner, Richard A., Mijan F. Huq, and Deirdre Shoemaker.
        "Initial data and coordinates for multiple black hole systems."
        Physical Review D 59.2 (1998): 024015.
    """
    H = self.H(point)
    llower = self.llower(point)

    return np.identity(3) + np.array([
        [2.0 * H * llower[i] * llower[j] for j in range(3)]
        for i in range(3)
    ])
```

Note that we defined helper methods ```H(point)``` and ```llower(point)``` in the ```KerrKSSlice``` class for the $H$ function and the $l_{\mu}$ vector above. These helper methods are not shown above for clarity, but can be found in the file ```distorted-motsfinder-public/motsfinder/metric/analytical/kerrks.py```. 

If you have variables in your analytical metric which are defined in terms of other variables and will be referenced often:

### It is strongly recommended that you define helper methods to avoid redundant code and potential bugs.

## d) Define derivatives of the 3-metric in ```diff```

If you have analytic expressions for the 1st, 2nd, etc. derivatives of 3 metric, you can enter them in the method ```diff```, similar to how the metric was entered. See ```schwarzschildks.py``` for a specific example of how to enter the derivatives; further documentation can be found for methods by looking at documentation on parent classes, etc.

Otherwise, specify that the 0th derivative case should simply return your metric, and raise a ```NotImplementedError``` for any other derivatives, as in the following example for the Kerr case (code below can be copied and pasted into your ```MetricXSlice``` class if you do not have analytic expressions for nonzero derivatives of the 3-metric):

```python
def diff(self, point, inverse=False, diff=1):
    if inverse:
        return self._compute_inverse_diff(point, diff=diff)
    if diff == 0:
        return self._mat_at(point)

    raise NotImplementedError
```

# Step 4: Specify the ```get``` methods

In each of the ```get``` methods following the ```diff``` method (i.e. ```get_curv```, ```get_lapse```, ```get_dtlapse```, ```get_shift```, ```get_dtshift```), specify the classes that will be returned when they are called. For example, for the ```get_curv``` method, for your ```metricX``` metric, you can return ```MetricXSliceCurv(self)```, which you must write eventually. We show how to define these classes below, using Kerr as an example. It is recommended that you include the metric name in the class returned, for clarity.

All notation follows [1], unless otherwise stated, and we use the quantities defined previously.


## a) (i) Define the lapse, $\alpha$

For Kerr in Kerr-Schild, the lapse is $$\alpha = \frac{1}{\sqrt{1 + 2H}}$$

In code, this is:

```python
class KerrKSSliceLapse():
    r"""Lapse function of the Kerr slice in Kerr-Schild form.

    This is eq. (13) of [1].

    @b References

    [1] Matzner, Richard A., Mijan F. Huq, and Deirdre Shoemaker. "Initial
        data and coordinates for multiple black hole systems." Physical Review
        D 59.2 (1998): 024015.
    """
    def __init__(self, metric):
        self._g = metric

    def __call__(self, point, diff=0):
        if diff != 0:
            raise NotImplementedError

        H = self._g.H(point)

        return 1.0 / np.sqrt(1.0 + 2.0*H)
```

Note that we are using the ```H(point)``` helper method defined in the ```MetricXSlice``` class. To access that helper method, we write ```self._g.H(point)```.

The ```get_lapse``` method in the ```KerrKSSlice``` class would then become:

```python
def get_lapse(self):
    return KerrKSSliceLapse(self)
```

### NOTE:

If you are planning to use MOTSFinder according to Usage 1, the lapse function is not required (since it's simply a gauge choice). For simplicity, you can return the ```trivial_lapse``` method, which sets the lapse to ```1.0```. To do so, make sure to specify ```trivial_lapse``` in your ```import``` statements at the top of the file like so:

```python
from ..base import trivial_lapse
```

or, equivalently, like this, with the existing ```import```s:

```python 
from ..base import _ThreeMetric, trivial_lapse, trivial_dtlapse, trivial_dtshift
```

Then, you can write the following (in the ```MetricXSlice``` class):

```python
def get_lapse(self):
    return trivial_lapse

```

# a) (ii) Define the time derivative of the lapse

If you have analytic expression for the time derivative of the lapse, you can enter the expression as:

```python
class MetricXSliceDtlapse():
    def __init__(self, metric):
        self._g = metric

    def __call__(self, point, diff=0):
        # ... as for MetricXSliceLapse but for time derivative of lapse 
    
        return ...
```
and write the following in the ```MetricXSlice``` class:

```python
def get_dtlapse(self):
    return MetricXSliceDtlapse(self)
```

### NOTE:

If you do not define a ```get_dtlapse``` method, a warning will be given when run, stating that the time derivative of the lapse is not given, and will be using zero. 

If the time derivative is 0, as in the case for Kerr, or you are planning to use MOTSFinder according to Usage 1, you can return ```trivial_dtlapse```, which sets the time derivative to ```0.0```. To do so, make sure ```trivial_dtlapse``` is specified in your ```import``` statements at the top of the file like so:

```python
from ..base import trivial_dtlapse
```

or, equivalently, like this, with the existing ```import```s:

```python 
from ..base import _ThreeMetric, trivial_dtlapse, trivial_dtshift
```

Then, you can write the following (in the ```MetricXSlice``` class):

```python
def get_dtlapse(self):
    return trivial_dtlapse
```

## b) (i) Define the shift, $\beta^i$

For Kerr in Kerr-Schild, the shift is $$\beta_i = 2Hl_i$$

Note that the shift in MOTSFinder refers to the shift in vector form, so we must raise the index. This can be done in the code, using ```np.einsum(...)```:

```python
class KerrKSSliceShift():
    r"""Shift vector field of the Kerr slice in Kerr-Schild form.

    This is the raised version of eq. (14) of [1].

    @b References

    [1] Matzner, Richard A., Mijan F. Huq, and Deirdre Shoemaker. "Initial
        data and coordinates for multiple black hole systems." Physical Review
        D 59.2 (1998): 024015.
    """
    def __init__(self, metric):
        self._g = metric

    def __call__(self, point, diff=0):
        if diff != 0:
            raise NotImplementedError

        H = self._g.H(point)
        llower = self._g.llower(point)

        betal = np.asarray([2.0 * H * llower[i] for i in range(3)]) # eq. (14) of [1]
        g_inv = self._g.diff(point, inverse=True, diff=0)
        beta = np.einsum('ij,j->i', g_inv, betal)
        return beta
```

The ```get_shift``` method in the ```KerrKSSlice``` class would then become:

```python
def get_shift(self):
    return KerrKSSliceShift(self)
```

### NOTE:

If you are planning to use MOTSFinder according to Usage 1, the shift vector is not required (since it's simply a gauge choice). For simplicity, you can return ```trivial_shift```, which sets the shift vector field to ```0.0```. To do so, make sure ```trivial_shift``` is specified in your ```import``` statements at the top of the file like so:

```python
from ..base import trivial_shift
```

or, equivalently, like this, with the existing ```import```s:

```python 
from ..base import _ThreeMetric, trivial_shift, trivial_dtlapse, trivial_dtshift
```

Then, you can write the following (in the ```MetricXSlice``` class):

```python
def get_shift(self):
    return trivial_shift
```

# b) (ii) Define the time derivative of the shift vector

If you have analytic expression for the time derivative of the shift, you can enter the expression as:

```python
class MetricXSliceDtshift():
    def __init__(self, metric):
        self._g = metric

    def __call__(self, point, diff=0):
        # ... as for MetricXSliceShift but for time derivative of shift 
        
        return ...
```
and write the following in the ```MetricXSlice``` class:

```python
def get_dtshift(self):
    return MetricXSliceDtshift(self)
```

### NOTE:
If you do not define a ```get_dtshift``` method, a warning stating that the time derivative of the shift is not given, and will be using zero, will be given when run. 

If the time derivative is the 0 vector, as in the case for Kerr, or you are planning to use MOTSFinder according to Usage 1, you can return ```trivial_dtshift```, which sets the time derivative of the shift vector field to ```0.0```. To do so, make sure ```trivial_dtshift``` is specified in your ```import``` statements at the top of the file like so:

```python
from ..base import trivial_dtshift
```

or, equivalently, like this, with the existing ```import```s:

```python 
from ..base import _ThreeMetric, trivial_dtlapse, trivial_dtshift
```

Then, you can write the following (in the ```MetricXSlice``` class):

```python
def get_dtshift(self):
    return trivial_dtshift
```

## c) Define the extrinsic curvature, $K_{ij}$

For Kerr in Cartesian Kerr-Schild form, the extrinsic curvature takes the form: 

$$ K_{ij} = -\alpha\left[2Hl^c(Hl_il_j)_{,c} - (Hl_i)_{,j} - (Hl_j)_{,i}\right]$$

where $$l^{\mu} = \left(1, -\frac{rx+ay}{r^2 + a^2}, -\frac{ry - ax}{r^2 + a^2}, -\frac{z}{r}\right)$$

Note that $K_{ij} = K_{ji}$. In addition, the difference in sign convention between us and [2] is taken into account here. See ```distorted-motsfinder-public/motsfinder/metric/analytical/kerrks.py``` for explicit details on implementation. We comment here that we calculate $K_{ij}$ in small chunks rather than one long expression; i.e. we calculate the derivatives of $H$ and $l_{i}$, then the contraction, then the entire sum. 

```KerrKSSliceCurv()``` returns a 3x3 matrix with the curvature components, so you must have expressions for each of the 9 curvature components. 

After defining ```KerrKSSliceCurv()```, you can define ```get_curve()``` in the ```KerrKSSlice``` class as:

```python
def get_curv(self):
    return KerrKSSliceCurv(self)
```


You have successfully implemented the ```metricX``` metric upon completion of this step.

# Step 5: Add to ```reloading.py```

To use the newly implemented ```metricX``` metric, the reloader must be aware of it:

## a) Open the file ```distorted-motsfinder-public/motsfinder/ipyutils/reloading.py```

## b) Search for ```"motsfinder.metric.analytical.schwarzschildks"```

## c) Below this line, add ```"motsfinder.metric.analytical.metricX"```, separated from the next entry by a comma

- The line added should follow the structure of the path to ```metricX.py```, relative to the ```motsfinder``` folder (i.e. ```motsfinder/metric/analytical/metricX.py```). However, in ```reloading.py```, the ```/``` in the path must be replaced with ```.```, and the file extension must not be included.

